In [ ]:
!pip install tokenizers
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")

In [ ]:
def find_perplexity(text_1, text_2):
    input_ids1 = tokenizer(text_1, return_tensors='pt')['input_ids']
    input_ids2 = tokenizer(text_2, return_tensors='pt')['input_ids']

    with torch.no_grad():
        loss_1 = model(input_ids=input_ids1, labels=input_ids1).loss
        loss_2 = model(input_ids=input_ids2, labels=input_ids2).loss

    perplexity_1 = np.exp(loss_1)
    perplexity_2 = np.exp(loss_2) 

    if perplexity_1 < perplexity_2:
        model_choice = 'toxic original'
    else: 
        model_choice = 'API non-toxic version'

    return perplexity_1, perplexity_2, model_choice


In [ ]:
data = pd.read_csv('rubert.csv')
original_dialogues = [x+' '+y for x,y in zip(data['Реплика 1 toxicity'], data['Реплика 2 toxicity'])]
API_dialogues = [x+' '+y for x,y in zip(data['Реплика 1 toxicity'], data['Реплика 2 original dialogue 1'])]

In [ ]:
perplexities_1 = []
perplexities_2 = []
model_choices = []


for text_1, text_2 in tqdm(zip(original_dialogues, API_dialogues)):
    perplexity_1, perplexity_2, model_choice = find_perplexity(text_1, text_2)
    perplexities_1.append(perplexity_1)
    perplexities_2.append(perplexity_2)
    model_choices.append(model_choice)

data['perplexity_original_toxic'] = perplexities_1
data['perplexity_API_non_toxic'] = perplexities_2
data['GPT_choice'] = model_choices

0it [00:00, ?it/s]

In [ ]:
data.to_csv('rubert_gpt3_balanced.csv')